In [1]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
import random

В этой задаче нужно сдлеать SGD версию нашей модели 

Добавьте в класс MyLineReg два новых параметра:

`sgd_sample` – кол-во образцов, которое будет использоваться на каждой итерации обучения. Может принимать либо целые числа, либо дробные от 0.0 до 1.0.
По-умолчанию: None

`random_state` – для воспроизводимости результата зафиксируем сид (об этом далее).
По-умолчанию: 42

Внесем изменение в алгоритм обучения:

В начале обучения фиксируем сид (см. ниже).

В начале каждого шага формируется новый мини-пакет, состоящий из случайно выбранных элементов обучающего набора. 
Кол-во отобранных элементов определяется параметром sgd_sample:

`Если задано целое число, то из исходного датасета берется ровно столько примеров сколько указано.`
    
`Если задано дробное число, то рассматриваем его как долю от количества строк в исходном датасете (округленное до целого числа).`
    
Расчет градиента (и последующее изменение весов) делаем на основе мини-пакета.

Все остальные параметры, если они заданы (например, регуляризация), также должны учитываться при обучении.

Ошибку и метрику необходимо считать на всем датасете, а не на мини-пакете.

Если `sgd_sample = None`, то обучение выполняется как раньше (на всех данных).

`Случайная генерация`

Т.к. у нас формальная проверка кода, то у всех должны получиться одинаковые случайные подвыборки. Поэтому и способ у всех будет одинаковый.

В начале обучения посредством модуля random фиксируем сид:

random.seed(<random_state>)
В начале каждой итерации сформируем порядковые номера строк, которые стоит отобрать.

sample_rows_idx = random.sample(range(X.shape[0]), <sgd_sample>)
В этом случае при каждом запуске будут генерироваться одни и те же номера строк. Что позволит нам добиться воспроизводимости.

З.Ы. Модуль random уже импортирован.
З.Ы.2. При отборе строк не стоит полагаться на индекс пандаса – он может быть не последовательным.



In [4]:
X, y = make_regression(n_samples=600, n_features=5, n_informative=8, noise=15, random_state=41)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4', 'f5'])
y = pd.Series(y, name='target')

In [5]:
X

,f1,f2,f3,f4,f5
0,-0.986257,1.745565,-1.200874,-0.047218,0.267876
1,2.015698,0.488657,0.996674,0.653400,0.065277
2,1.701293,-1.477120,2.039221,-0.025737,0.205535
3,-0.301803,-1.172736,0.528437,0.770789,0.158079
4,1.176931,0.894768,-0.556505,-0.974740,-1.182949
...,...,...,...,...,...
595,-1.451426,-0.050116,-0.249363,-0.384609,0.653114
596,-0.482776,-0.501015,0.504170,-1.131012,0.970515
597,1.880913,-0.866513,0.057113,-0.284894,1.084699
598,1.116665,-0.965840,-1.428672,0.140047,1.800045


In [6]:
range(X.shape[0])

range(0, 600)

In [8]:
random.seed(34)
idx = random.sample(range(X.shape[0]), 4)
idx

[541, 365, 599, 28]

In [9]:
X.iloc[idx]

,f1,f2,f3,f4,f5
541,-1.945076,-0.116411,0.291020,-1.012940,1.174804
365,1.038227,-0.081436,1.257007,-0.514092,-0.004470
599,0.217178,0.540150,-0.008538,1.607709,-0.472818
28,0.577252,-0.338238,0.464582,1.212734,-1.456254


In [16]:
y.iloc[idx]

541   -220.245257
365    198.816743
599    148.894350
28      99.717342
Name: target, dtype: float64

In [18]:
data = pd.concat([X, y], axis=1)

In [20]:
data.iloc[idx]

,f1,f2,f3,f4,f5,target
541,-1.945076,-0.116411,0.291020,-1.012940,1.174804,-220.245257
365,1.038227,-0.081436,1.257007,-0.514092,-0.004470,198.816743
599,0.217178,0.540150,-0.008538,1.607709,-0.472818,148.894350
28,0.577252,-0.338238,0.464582,1.212734,-1.456254,99.717342


In [22]:
def get_sgd_sample(n_rows: int, sgd_sample):
        """Вернет Батчи."""
        if isinstance(sgd_sample, int):
            return sgd_sample
        elif isinstance(sgd_sample, float):
            return max(1, int(sgd_sample * n_rows)) 

In [24]:
random.seed(1)
for i in range(5):
    sample_size = get_sgd_sample(X.shape[0], 0.02)
    print(random.sample(range(X.shape[0]), sample_size))

[137, 582, 64, 261, 120, 507, 460, 483, 388, 214, 96, 499]
[29, 399, 443, 2, 456, 272, 234, 104, 325, 31, 22, 26]
[554, 9, 390, 221, 432, 29, 540, 227, 448, 507, 566, 238]
[353, 236, 224, 470, 296, 22, 426, 569, 102, 190, 303, 123]
[340, 512, 432, 519, 194, 310, 290, 511, 517, 402, 35, 491]


In [26]:
sample_size

12

In [28]:
random.sample(range(600), sample_size)

[248, 413, 424, 177, 375, 561, 383, 88, 449, 520, 110, 167]

In [30]:
X.shape[0]

600

In [32]:
'''
        # Напишем цикл обучения
        for i in range(self.n_iter):
            pred = X_copy.dot(w)                         # считаем предсказания модели
            error = pred - y                             # вычислим ошибку (предсказания - реальные значения)
            grad = (2/len(y)) * np.dot(error, X_copy)    # вычислим градиент по весам модели
            grad = self.__apply_regularization(grad, w)  # добавим регуляризацию, если она задана

            if self.metric:                              # Если задана метрика, то считаем её и обновляем self.best_score
                self.best_score = self.__calculate_metric(pred=pred, y_true=y, metric=self.metric)
            
            if verbose and (i % 10 == 0 or i == self.n_iter - 1): # Eсли задано логирование, то выводим логи
                self.__log_training_step(i, X_copy, y, w)
                
            lr = self.__get_learning_rate(i+1)             # проверяем на динамичность lr
            w -= lr* grad                                  # делаем шаг обучения
        self.weights = w    # сохроняем веса
''';

In [40]:
class MyLineReg():
    def __init__(self, n_iter=100, learning_rate=0.1, reg=None, l1_coef=0, l2_coef=0, random_state=42, sgd_sample=None, metric=None, weights=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.sgd_sample = sgd_sample
        self.random_state = random_state
        self.weights = weights
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.best_score = None

        self.__validate_params() # проверка входных параметров

    def __str__(self):
        return f'MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'
        
    def __validate_params(self):  # Проверяем на корректность входные параметры 
        # Проверям на входные параметры подсчета метрики
        if self.metric is not None and self.metric not in ['mae', 'mse', 'rmse', 'mape', 'r2']:
            raise ValueError("metric must be 'mae', 'mse', 'rmse', 'mape' or 'r2'")
        # Проверям на входные параметры регулиризации 
        if self.reg is not None and self.reg not in ['l1', 'l2', 'elasticnet']:
            raise ValueError("reg must be 'l1', 'l2', or 'elasticnet'")
        # Проверка коэффициентов (должны быть от 0 до 1)
        if self.l1_coef < 0 or self.l1_coef > 1:
            raise ValueError("l1_coef must be in [0, 1]")
        if self.l2_coef < 0 or self.l2_coef > 1:
            raise ValueError("l2_coef must be in [0, 1]")
        # Проверка входных параметров для SGD
        if self.sgd_sample is not None:
            if isinstance(self.sgd_sample, float) and (self.sgd_sample <= 0 or self.sgd_sample > 1):
                raise ValueError("sgd_sample as float must be in (0, 1]")
            if isinstance(self.sgd_sample, int) and (self.sgd_sample <= 0):
                raise ValueError("sgd_sample as int must be > 0")


    # Напишем метод с приватным модификатором доступа для подсчета разнных метрик 
    def __calculate_metric(self, pred: np.array, y_true: np.array, metric: str) -> float:
        """
        Вычисляет значение метрики.

        Параметры:
        pred (np.array): Предсказанные значения.
        y_true (np.array): Фактические значения.
        metric (str): Название метрики.

        Возвращает:
        float: Значение метрики.
        """
        if metric == 'mae':
            return np.mean(np.abs(pred - y_true))
        elif metric == 'mse':
            return np.mean((pred - y_true)**2)
        elif metric == 'rmse':
            return np.sqrt(np.mean((pred - y_true)**2))
        elif metric == 'mape':
            return 100 * np.mean(np.abs((pred - y_true) / y_true))
        elif metric == 'r2':
            ss_residual = np.sum((pred - y_true)**2)
            ss_total = np.sum((y_true - np.mean(y_true))**2)
            return 1 - (ss_residual / ss_total)
        else:
            raise ValueError(f"Unknown metric: {metric}")
            
    def __calculate_loss(self, X: np.array, y: np.array, w: np.array) -> float:       
        '''
        Вычисляем общий loss (MSE + регуляризация).
        X: np.array (Матрица признаков)
        y: np.array (Вектор значений)
        w: np.array (Вектор весов)
        '''
        error = np.dot(X,w) - y
        MSE = np.mean(error**2) 
        
        # Добавим регуляризацию, если она установлена
        if self.reg == 'l1':
            return MSE + self.l1_coef * np.sum(np.abs(w))
        elif self.reg == 'l2':
            return MSE + self.l2_coef * np.sum(w**2)
        elif self.reg == 'elasticnet':
            return MSE + self.l1_coef * np.sum(np.abs(w)) + self.l2_coef * np.sum(w**2)
        else:
            return MSE

    def __log_training_step(self, iteration: int, X: np.array, y: np.array, w: np.array):
        """Логирует процесс обучения."""
        loss = self.__calculate_loss(X, y, w)
        if self.metric:
            metric_value = self.__calculate_metric(X.dot(w), y, self.metric)
            print(f'Iteration {iteration} | loss: {loss:.2f} | {self.metric}: {metric_value:.2f}')
        else:
            print(f'Iteration {iteration} | loss: {loss:.2f}')

    def __apply_regularization(self, grad: np.array, w: np.array) -> np.array:
        """Добавляет градиент регуляризации."""
        if self.reg == 'l1':
            grad += self.l1_coef * np.sign(w)
        elif self.reg == 'l2':
            grad += 2 * self.l2_coef * w
        elif self.reg == 'elasticnet':
            grad += self.l1_coef * np.sign(w) + 2 * self.l2_coef * w
        return grad

    def __get_learning_rate(self, iteration: int) -> float:
        """Вычисляет learning_rate (статический или динамический)."""
        if callable(self.learning_rate):
            return self.learning_rate(iteration)
        else:
            return self.learning_rate

    def __get_sgd_sample(self, n_rows: int, sgd_sample):
        """Вернет размер Батча."""
        if isinstance(sgd_sample, int):                   # если целое число элементов (в батче)
            return sgd_sample                             # веренем ровно столько экзмпляров в одном батче
        elif isinstance(sgd_sample, float):               # если sgd_sample долями задан
            return max(1, int(sgd_sample * n_rows))       # вернем долю это числа элементов или 1, если за долю взята очень маленькое число
        
    def fit(self, X: pd.DataFrame, y:pd.Series, verbose=False):
        """
        Обучение модели линейной регрессии.
        Параметры:
        
        X (pd.DataFrame): Матрица признаков.
        
        y (pd.Series): Вектор целевых значений.
        
        verbose (bool): Флаг для вывода информации о процессе обучения.
        """
        random.seed(self.random_state)                           # зафиксируем seed
        X_copy = X.copy()                                        # создадим копию нашей матрицы признаков, что бы не изменить изначальный
        X_copy.insert(0, 'base', 1)                              # допишем слева столбик из 1 для свободного члела
        n_rows = X_copy.shape[0]                                 # зафиксируем число строк в матрице признаков
        
        w = np.ones(X_copy.shape[1])                             # составим вектор весов, заполненный из 1
        
        if verbose:                                              # вычеслим loss на старте обучения
            loss = self.__calculate_loss(X_copy, y, w) # вычисляем loss
            if self.metric is None:
                print(f'start | loss: {loss:.2f}')
            else:
                metric = self.__calculate_metric(pred=X_copy.dot(w), y_true=y, metric=self.metric)
                print(f'start | loss: {loss:0.2f} | {self.metric}: {metric:0.2f}')

        # Напишем цикл обучения 
        for i in range(self.n_iter):
            if self.sgd_sample is not None:                                   # если задано SGD 
                sample_size = self.__get_sgd_sample(n_rows, self.sgd_sample)  # cчитаем сколько объектов должно быть в одном батче
                sample_idx = random.sample(range(n_rows), sample_size)        # получаем индексы для этих объектов 
                X_batch = X_copy.iloc[sample_idx]                             # вернем значения этих объектов для матрицы признаков
                y_batch = y.iloc[sample_idx]                                  # вернем значения этих объектов для вектора таргет
            else:                           
                X_batch = X_copy             #  если SGD не задан просто в батчи кладем все наши данные          
                y_batch = y
                
            pred = X_batch.dot(w)                                             # считаем предсказания модели
            error = pred - y_batch                                            # вычислим ошибку (предсказания - реальные значения)
            grad = (2/len(y_batch)) * np.dot(error, X_batch)                  # градиент считаем по батчу (если он задан), если нет на всех данных
            grad = self.__apply_regularization(grad, w)                       # добавим регуляризацию, если она задана

            if self.metric:                                                   # Если задана метрика, то считаем её и обновляем self.best_score
                # метрику считаем на всем датасете, а не на батчах
                self.best_score = self.__calculate_metric(pred=X_copy.dot(w), y_true=y, metric=self.metric)

            if verbose and (i % 10 == 0 or i == self.n_iter - 1):             # Если задано логирование, то выводим логи
                self.__log_training_step(i, X_copy, y, w)                     # ошибку считаем на всем датасете, а не на мини-пакете

            lr = self.__get_learning_rate(i+1)                                # проверяем на динамичность lr
            w -= lr * grad                                                    # делаем шаг обучения
            
        self.weights = w  # сохроняем веса
        
    def predict(self, X: pd.DataFrame)->np.array:
        """
        Предсказание целевых значений.

        Параметры:
        X (pd.DataFrame): Матрица признаков.

        Возвращает:
        np.array: Предсказанные значения.
        """
        X_copy = X.copy()            # делаем копию матрицы фичей, чтобы не менять изначальный датафрейм
        X_copy.insert(0, 'base', 1)  # допишем слева столбик из 1 для свободного члена
        predict = X_copy.dot(self.weights) # делаем предсказания X_copy @ self.weights
        return predict 

    def get_best_score(self):  # Метод для вывода лучшего качетва на выбранной метрике
        if self.metric is None:
            raise ValueError("Metric was not set during model initialization.")
        return self.best_score

    def get_coef(self):
        return self.weights[1:] # метод для вывода весов начиная с 1го заначения            


In [43]:
model = MyLineReg(sgd_sample=65, metric='mse', reg='elasticnet', l1_coef=0.2, l2_coef=0.3)

In [45]:
model.fit(X, y, verbose=True)

start | loss: 33314.28 | mse: 33311.28
Iteration 0 | loss: 33314.28 | mse: 33311.28
Iteration 10 | loss: 7855.21 | mse: 2702.99
Iteration 20 | loss: 7791.48 | mse: 1619.33
Iteration 30 | loss: 7751.32 | mse: 1894.90
Iteration 40 | loss: 7755.35 | mse: 1903.18
Iteration 50 | loss: 7757.71 | mse: 1718.75
Iteration 60 | loss: 7767.80 | mse: 2077.50
Iteration 70 | loss: 7764.69 | mse: 1932.31
Iteration 80 | loss: 7771.32 | mse: 1665.40
Iteration 90 | loss: 7771.52 | mse: 1701.74
Iteration 99 | loss: 7779.28 | mse: 1519.30


In [48]:
model.get_coef()

array([78.05210484, 77.38194516, 78.35103279, 40.60419614,  8.38999286])

In [50]:
model.weights

array([ 0.88120605, 78.05210484, 77.38194516, 78.35103279, 40.60419614,
        8.38999286])